# Ejercicio 10

Toma tu número $p$ de la lista publicada para este ejercicio. 

1. Calcula el símbolo de Jacobi $\left(\frac{-11}{p}\right)$. Si sale $1$, usa el algoritmo de Tonelli-Shanks para hallar las soluciones a la congruencia $x^2 \equiv -11 \pmod{p}$.
2. Usa una de esas soluciones para factorizar el ideal principal $(p) = \left(p, n + \sqrt{-11}\right)\left(p, n - \sqrt{-11}\right)$ como producto de dos ideales.
3. Aplica el algoritmo de Cornachia-Smith modificado a $2p$ y $n$ para encontrar una solución a la ecuación diofántica $4p = x^2 + 11y^2$, y úsala para encontrar una factorización de $p$ en a.e. del cuerpo $\mathbb{Q}\left[\sqrt{p}\right]$.
4. ¿Son ideales principales tus ideales $\left(p, n + \sqrt{-11}\right)$ y $\left(p, n - \sqrt{-11}\right)$?

In [1]:
p = 77432081
a = -11

## Apartado 1

In [2]:
jacobi_symbol(-11, p)

1

(comprobaciones de que p es primo en julia)

In [3]:
p.mod(8)

1

Como $p \equiv 1 \pmod{8}$, tenemos que usar el algoritmo especial.

$$p - 1 = 2^4\cdot 5 \cdot 11 \cdot 87991 = 2^4 \cdot 4839505\\ $$

In [7]:
q = 4839505
e = 4-1

Elegimos ahora un $n$ que no sea residuo cuadrático:

In [8]:
n = 1
while jacobi_symbol(n, p) != -1:
    n = n + 1 
n

3

Hemos obtenido un generador del 2-subgrupo de Sylow $G \cong \mathbb{Z}_{2^n} = \mathbb{Z}_{2^3}$.

Solo nos queda aplicar Tonelli-Shanks para encontrar las soluciones de $x^2 \equiv -11 \pmod{p}$

In [7]:
def orden(t, p):
    for i in range(1, p):
        if t.powermod(i, p) == 1:
            return i
    
    return 0

        
def calcular_z(p):
    n = 1
    while jacobi_symbol(n, p) != -1:
        n = n + 1 
    
    e = factor(p-1)[0][1]
    q = (p-1)//2^e
    z = (n^q).mod(p)
    
    return z
    

def tonelli_shanks(a, p):
    e = factor(p-1)[0][1]
    print("e = " + str(e))
    q = (p-1)//2^e
    print("q = " + str(q))
    t = a^q
    print("t mod p = " + str(t.mod(p)))
    z = calcular_z(p)
    print("z mod p = " + str(z))
    r = a.powermod((q+1)//2, p)
    print("r mod p = " + str(r))
    r_1 = r
    
    while t.mod(p) != 1:
        print("\nInicio de bucle:")
        i = factor(orden(t, p))[0][1]
        print("\tOrden de t = " + str(orden(t, p)) + " => " + "i = " + str(i))
        b = z.powermod(2^(e - i - 1), p)
        print("\tb mod p = " + str(b))
        b_cuadrado = b.powermod(2, p)
        print("\tb^2 mod p = " + str(b_cuadrado))
        r_1 = (r_1 * b).mod(p)
        print("\tr_1 mod p = " + str(r_1))
        t_1 = (t * b_cuadrado).mod(p)
        print("\tt_1 mod p = " + str(t_1))

        t = t_1
    
    x_1 = r_1.mod(p)
    x_2 = (p - x_1).mod(p)
    
    return [x_1, x_2]

soluciones = tonelli_shanks(a, p)
soluciones

e = 4
q = 4839505
t mod p = 8112714
z mod p = 77354166
r mod p = 14183777

Inicio de bucle:
	Orden de t = 8 => i = 3
	b mod p = 77354166
	b^2 mod p = 31044907
	r_1 mod p = 59107158
	t_1 mod p = 1


[59107158, 18324923]

Podemos comprobar que, en efecto, son soluciones:

In [85]:
print("p - 11 = " + str(p + a))
print("¿x^2 === -11 mod p?")
for sol in soluciones:
    print(sol.powermod(2, p))

p - 11 = 77432070
¿x^2 === -11 mod p?
77432070
77432070


## Apartado 2

Consideramos la solución impar $n = 18324923$.

In [86]:
n = 18324923

Puesto que $p$ es un primo impar y no divide a $a = -11$, entonces se tiene la descomposición

$$
\begin{aligned}
(p) & = \left(p, n + \sqrt{a}\right)\left(p, n - \sqrt{a}\right) = \\ 
    & = \left(77432081, 18324923 + \sqrt{-11}\right)\left(77432081, 18324923 - \sqrt{-11}\right)
\end{aligned}
$$

## Apartado 3

Aplicando el algoritmo de Cornacchia-Smith modificado, obtenemos

In [153]:
def cornacchia_smith_modificado(p, n):
    siguiente_Div = 2*p
    siguiente_div = n
    
    resto = 2*p
    i = 1
    
    while not resto < 2*sqrt(p):
        cociente, resto = divmod(siguiente_Div, siguiente_div)
        print("[" + str(i) + "] " + str(siguiente_Div) + " = " + str(cociente) + "*" + str(siguiente_div) + " + " + str(resto))

        siguiente_Div = siguiente_div
        siguiente_div = resto
        i = i + 1
        
    return resto    
    
a = cornacchia_smith_modificado(p, n)
a


[1] 154864162 = 8*18324923 + 8264778
[2] 18324923 = 2*8264778 + 1795367
[3] 8264778 = 4*1795367 + 1083310
[4] 1795367 = 1*1083310 + 712057
[5] 1083310 = 1*712057 + 371253
[6] 712057 = 1*371253 + 340804
[7] 371253 = 1*340804 + 30449
[8] 340804 = 11*30449 + 5865


5865

Tras $8$ divisiones, obtenemos el resto $x = 5865$. Si despejamos $y$ de la ecuación $4p = x^2 + 11y^2$, obtenemos 

$$
y = \sqrt{\frac{4p - x^2}{11}}
$$

In [161]:
b = sqrt((4*p - a^2)/11)
b

5003

In [163]:
4*p == a^2 + 11 * b^2

True

Esto nos da la factorización de $\mathbb{Q}\left[\sqrt{p}\right]$

$$
\begin{aligned}
& 4p = 5865^2 + 11*5003^2 \Rightarrow \\
p & = \left(\frac{a + b\sqrt{-11}}{2}\right)\left(\frac{a - b\sqrt{-11}}{2}\right) = \\
  & = \left(\frac{5865 + 5003\sqrt{-11}}{2}\right)\left(\frac{5865 - 5003\sqrt{-11}}{2}\right)
\end{aligned}
$$

## Apartado 4

Los ideales obtenidos son principales, puesto que aplicando el teorema 17, se tiene que $m = -11 \Rightarrow \mathbb{Q}\left[\sqrt{-11}\right]$  es DIP.